In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Function to load CSV files
def load_csv(file_path):
    columns = ['ID', 'Component', 'Metric', 'Value']
    data = pd.read_csv(file_path, header=None, names=columns)
    data['Value'] = pd.to_numeric(data['Value'], errors='coerce')
    data = data.dropna(subset=['Value'])
    
    # Normalize component names (e.g., L1VCache[x] -> L1VCache, L2[x] -> L2)
    data['Component'] = data['Component'].str.replace(r'\[\d+\]', '', regex=True)
    data['CombinedMetric'] = data['Component'] + ' - ' + data['Metric']
    return data

# Path selection widgets
file1_path = widgets.Text(value='', description='Normal Path:')
file2_path = widgets.Text(value='', description='Prefetch Path:')
load_button = widgets.Button(description='Load Files')

# Dropdown for metric selection
metric_dropdown = widgets.Dropdown(options=[], description='Metric:')

# Output widget
output = widgets.Output()

# Load data function
def load_data(_):
    output.clear_output()
    try:
        data1 = load_csv(file1_path.value)
        data2 = load_csv(file2_path.value)
        common_metrics = set(data1['CombinedMetric'].unique()).intersection(set(data2['CombinedMetric'].unique()))
        metric_dropdown.options = sorted(common_metrics)
    except Exception as e:
        with output:
            print(f"Error loading files: {e}")

# Compute averages
def compute_averages(metric):
    output.clear_output()
    try:
        data1 = load_csv(file1_path.value)
        data2 = load_csv(file2_path.value)
        avg1 = data1[data1['CombinedMetric'] == metric]['Value'].mean()
        avg2 = data2[data2['CombinedMetric'] == metric]['Value'].mean()
        with output:
            print(f'Average Value for {metric}:')
            print(f'Normal: {avg1}')
            print(f'Prefetch: {avg2}')
    except Exception as e:
        with output:
            print(f"Error calculating averages: {e}")

# Event listeners
load_button.on_click(load_data)
metric_dropdown.observe(lambda change: compute_averages(change['new']), names='value')

# Display widgets
display(file1_path, file2_path, load_button, metric_dropdown, output)


Text(value='', description='Normal Path:')

Text(value='', description='Prefetch Path:')

Button(description='Load Files', style=ButtonStyle())

Dropdown(description='Metric:', options=(), value=None)

Output()